In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [79]:
data = pd.read_csv('/Users/alikhansainov/Desktop/Karpov/autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [80]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [81]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner
...,...,...,...,...,...,...,...
4335,Hyundai i20 Magna 1.4 CRDi (Diesel),2014,80000,Diesel,Individual,Manual,Second Owner
4336,Hyundai i20 Magna 1.4 CRDi,2014,80000,Diesel,Individual,Manual,Second Owner
4337,Maruti 800 AC BSIII,2009,83000,Petrol,Individual,Manual,Second Owner
4338,Hyundai Creta 1.6 CRDi SX Option,2016,90000,Diesel,Individual,Manual,First Owner


__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$



Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавая свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками.

Класс MeanTargetEncoderNoise должен иметь следующую сигнатуру:



In [149]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):
        
        self.categorical = categorical
        self.numeric = numeric
    
    def fit(self, X, y):

        X_fit = X.copy()
        y_fit = y.copy()
        X_with_target = pd.concat((X_fit, y_fit), axis=1)
        self.dict_of_means = {col : X_with_target.groupby(col)["selling_price"].mean()
                              for col in self.categorical}

        return self
        
    def transform(self, df):
        
        df_ = df.copy()
        for col in self.categorical:
                df_[col] = df_[col].map(self.dict_of_means[col])
                
                mean_value = self.dict_of_means[col].values.mean()
                df_[col] = df_[col].fillna(mean_value+(0.006*np.random.uniform(0, 1)))
        df_[self.categorical] = df_[self.categorical].astype('object')
        df_.fillna(0, inplace=True)
        
        return df_

In [93]:
data.dtypes

name             object
year              int64
selling_price     int64
km_driven         int64
fuel             object
seller_type      object
transmission     object
owner            object
dtype: object

In [94]:
numeric_cols = X.dtypes[X.dtypes != 'object'].index.tolist()
categorical_cols = X.dtypes[X.dtypes == 'object'].index.tolist()

In [148]:
np.random.seed(1)
encoder = MeanTargetEncoderNoise(categorical=categorical_cols, numeric=numeric_cols)
encoder.fit(X_train, y_train)
X_train_encoded = encoder.transform(X_train)
X_test_encoded = encoder.transform(X_test)

# Приведение категориальных колонок к типу object
X_train_encoded[categorical_cols] = X_train_encoded[categorical_cols].astype('object')
X_test_encoded[categorical_cols] = X_test_encoded[categorical_cols].astype('object')

X_train_encoded.head()
X_train_encoded.head(10).to_csv('X_train_encoded.csv', sep=';', index=False)


Разделите колонки на вещественные и категориальные. Приведите все категориальные колонки к типу `object`.

Далее применим наш кодировщик к `X_train, X_test`, так же как например мы применяем `StandardScaler`, чтобы проверить работоспособность нашего класса. Установите зерно датчика случайный чисел `np.random.seed(1)`.

После того, как вы изменили обучающую и тестовую выборки, сохраните первые 10 строк полученного промежуточного датафрейма обучающей выборки (`X_train`) в файл в формате csv с сепаратором `;`. Не забудьте индекс. Отправьте полученный файл в форму ниже.

Список колонок которые должны быть в файле для сдачи:
```py
cols = [
    "km_driven",
    "name",
    "year",
    "fuel",
    "seller_type",
    "transmission",
    "owner"
]
```

### Ваше решение


Разделение колонок на категориальные и числовые.

In [86]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


Реализация класса MeanTargetEncoderNoise.

In [170]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):
        
        self.categorical = categorical
        self.numeric = numeric
        self.C = 0.006
        
    
    def fit(self, X, y):

        X_fit = X.copy()
        y_fit = y.copy()
        X_with_target = pd.concat((X_fit, y_fit), axis=1)
        self.dict_of_means = {col : X_with_target.groupby(col)["selling_price"].mean()
                              for col in self.categorical}

        return self
        
    def transform(self, df):

        df_ = df.copy()
        for col in self.categorical:
            noise = 0.006 * np.random.uniform(0, 1, len(df_))
            df_[col] = df_[col].map(self.dict_of_means[col]) + noise
            df_[col] = df_[col].fillna(0)
        return df_

Проверка работы трансформера.

In [171]:

transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

save = train.head(10)
save.reset_index(drop=True).to_csv('/Users/alikhansainov/Desktop/Karpov/autos copy.csv', sep=';', index=False)


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [172]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]
np.random.seed(1)


### Your code is here
msle_depth_scores = []
for depth in max_depth_list:
    pipe = Pipeline([('custom_transformer',MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),('decisiontree', DecisionTreeRegressor(max_depth=depth))
                     ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    mse_pipe = mse(y_test, y_pred)
    msle_depth_scores.append(mse_pipe)
    print(f'Max_depth = {depth}, MSLE ={round(mse_pipe,2)}')

msle_samples_scores = []
for split in min_samples_split_list:
    pipe = Pipeline([('custom_transformer',MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),('decisiontree', DecisionTreeRegressor(min_samples_split=split))
                     ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    mse_pipe = mse(y_test, y_pred)
    msle_samples_scores.append(mse_pipe)
    print(f'Max_split = {split}, MSLE = {round(mse_pipe,2)}')

msle_impurity_scores = []
for impurity in min_impurity_decrease_list:
    pipe = Pipeline([('custom_transformer',MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),('decisiontree', DecisionTreeRegressor(min_impurity_decrease=impurity))
                     ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    mse_pipe = mse(y_test, y_pred)
    msle_impurity_scores.append(mse_pipe)
    print(f'Max_impurity = {impurity}, MSLE = {round(mse_pipe,2)}')

msle_leaf_scores = []
for leaf in max_leaf_nodes_list:
    pipe = Pipeline([('custom_transformer',MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),('decisiontree', DecisionTreeRegressor(max_leaf_nodes=leaf))
                     ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    mse_pipe = mse(y_test, y_pred)
    msle_leaf_scores.append(mse_pipe)
    print(f'Max_leaf_nodes = {leaf}, MSLE = {(mse_pipe)}')


# plt.fig(figsize=(10,6))
# plt.plot(max_depth_list, msle_scores, marker='o')
# plt.title('Relationship MSLE from parameter')
# plt.xlabel(max_depth_list)
# plt.ylabel(msle_scores)
# plt.show()


Max_depth = 3, MSLE =0.82
Max_depth = 5, MSLE =1.44
Max_depth = 8, MSLE =1.44
Max_depth = 12, MSLE =2.0
Max_split = 10, MSLE = 1.45
Max_split = 50, MSLE = 1.44
Max_split = 100, MSLE = 0.98
Max_split = 500, MSLE = 0.85
Max_impurity = 0, MSLE = 2.01
Max_impurity = 0.1, MSLE = 0.52
Max_impurity = 0.15, MSLE = 0.52
Max_impurity = 0.2, MSLE = 0.52
Max_leaf_nodes = 100, MSLE = 1.5132157597878306
Max_leaf_nodes = 200, MSLE = 1.9955219544405467
Max_leaf_nodes = 500, MSLE = 2.012521928992382


Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [173]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

### Your code is here

best_param_scores = []

pipe = Pipeline([('custom_transformer',MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),('decision_tree', DecisionTreeRegressor())
                     ])
search = GridSearchCV(pipe, param_grid=param_grid, scoring = 'neg_mean_squared_error')
search.fit(X_train, y_train)
best_params = search.best_params_
best_score = search.best_score_
print (f'Best param: {best_params}')
print (f'best score on test: {-search.score(X_test, y_test)}')


Best param: {'decision_tree__max_depth': 3, 'decision_tree__max_leaf_nodes': 200, 'decision_tree__min_impurity_decrease': 0.1, 'decision_tree__min_samples_split': 10}
best score on test: 0.520863439975766
